In [8]:
#Au lieu de OneHotEncoder, on utilise LabelEncoder
# Import label encoder 
from sklearn import preprocessing 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.base import BaseEstimator, ClassifierMixin
import optuna
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

In [9]:
#Chargement des données
df = pd.read_csv("Calendrier_et_resultats_Premier_League_clean.csv", index_col=['Date'])
df.index = pd.to_datetime(df.index)


In [10]:
clubs = list(set(df["Domicile"]))
print(clubs)
id_club = {club: i for i, club in enumerate(clubs)}
print(id_club)
df["Domicile"] = df["Domicile"].apply(lambda x: id_club[x])
#df["Extérieur"] = df["Extérieur"].apply(lambda x: id_club[x])


['Manchester Utd', 'Crystal Palace', 'Tottenham', 'Swansea City', 'Fulham', 'Stoke City', 'Arsenal', 'Luton Town', 'Sunderland', 'Chelsea', 'Watford', 'Aston Villa', 'Newcastle Utd', 'Brentford', 'West Brom', 'Huddersfield', 'Middlesbrough', 'Manchester City', 'Wolves', 'Liverpool', 'West Ham', 'Leicester City', 'Norwich City', 'Southampton', 'Leeds United', 'Sheffield Utd', 'Bournemouth', 'Brighton', 'Burnley', 'Cardiff City', "Nott'ham Forest", 'Ipswich Town', 'Everton', 'Hull City']
{'Manchester Utd': 0, 'Crystal Palace': 1, 'Tottenham': 2, 'Swansea City': 3, 'Fulham': 4, 'Stoke City': 5, 'Arsenal': 6, 'Luton Town': 7, 'Sunderland': 8, 'Chelsea': 9, 'Watford': 10, 'Aston Villa': 11, 'Newcastle Utd': 12, 'Brentford': 13, 'West Brom': 14, 'Huddersfield': 15, 'Middlesbrough': 16, 'Manchester City': 17, 'Wolves': 18, 'Liverpool': 19, 'West Ham': 20, 'Leicester City': 21, 'Norwich City': 22, 'Southampton': 23, 'Leeds United': 24, 'Sheffield Utd': 25, 'Bournemouth': 26, 'Brighton': 27, 'B

In [11]:
df['Extérieur'] = df['Extérieur'].apply(lambda x: id_club[x])

In [12]:
df.drop(["Arbitre",'Score','Notes','Affluence','xG','xG.1','Tribune'], axis=1, inplace=True)



In [13]:

#df = pd.get_dummies(df, columns=["Tribune"], drop_first=True, dtype=int)

In [14]:
df.head()

,Domicile,Extérieur,winner
Date,,,
2016-08-13,33,21,1
2016-08-13,28,3,-1
2016-08-13,23,10,0
2016-08-13,1,14,-1
2016-08-13,16,5,0


In [15]:
    
# Charger les données
data = df.copy()  # Remplacez par votre dataframe

# Préparation des données
X = data.drop(columns=['winner'])  # Remplacez 'winner' par le nom de la colonne cible
y = data['winner']

# Encodage de la variable cible (si nécessaire)
y = y.map({-1: 0, 0: 1, 1: 2})  # Exemple : -1 -> 0, 0 -> 1, 1 -> 2

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


def objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }

    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', **params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)


[I 2024-12-27 21:44:37,712] A new study created in memory with name: no-name-c190995b-50a2-405d-b939-93c23da161a6
C:\Users\elyan\AppData\Local\Temp\ipykernel_16192\3549194205.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\elyan\AppData\Local\Temp\ipykernel_16192\3549194205.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\elyan\AppData\Local\Temp\ipykernel_16192\3549194205.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0

Best hyperparameters:  {'learning_rate': 0.08995553273189885, 'max_depth': 10, 'n_estimators': 274, 'subsample': 0.8429252904301449, 'colsample_bytree': 0.9027124839896867, 'gamma': 0.38375891393426764, 'min_child_weight': 5}
Best accuracy:  0.5171339563862928


In [16]:
best_params = study.best_params


best_model = XGBClassifier( 
    eval_metric='mlogloss', 
    **best_params
)

# Entraîner le modèle sur les données d'entraînement
best_model.fit(X_train, y_train)


C:\Users\elyan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9027124839896867, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None,
              gamma=0.38375891393426764, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08995553273189885,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=274, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [17]:
df_test = pd.read_csv("Calendrier_et_resultats_Premier_League_test.csv", index_col=['Date'])
df_test.index = pd.to_datetime(df_test.index)

In [18]:
df_test["Domicile"] = df_test["Domicile"].apply(lambda x: id_club[x])
df_test["Extérieur"] = df_test["Extérieur"].apply(lambda x: id_club[x])

df_test.drop(["Arbitre",'Score','Notes','Affluence','xG','xG.1','Tribune'], axis=1, inplace=True)

#df_test = pd.get_dummies(df_test, columns=["Tribune"], drop_first=True, dtype=int)


In [19]:
y_pred = best_model.predict(df_test)

In [20]:
print(y_pred)

[2 2 0 2 2 0 0 2 1 2 0 2 2 2 2 0 2 0 2 0 2 1 2 0 2 2 0 2 2 2 0 2 2 0 2 0 2
 2 1 2 2 2 0 2 2 2 2 0 0 2 0 0 1 2 2 2 2 2 2 0 1 2 2 2 0 2 0 2 0 2 0 0 2 2
 2 2 2 2 2 2 2 0 0 0 0 2 1 0 2 2 2 2 0 2 0 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2
 0 0 2 0 2 2 2 0 2 2 2 2 2 0 0 2 2 2 2 2 0 0 2 2 0 1 2 1 2 2 0 2 2 0 2 2 2
 2 0 0 0 2 2 2 2 2 0 0 2 2 2 2 2 2 2 0 2 1 2 0 0 2 0 2 2 2 2 1 2 2 0 2 0 0
 2 2 2 1 2 2 2 0 2 2 2 0 2 2 0 0 2 2 2 2 0 0 2 2 2]
